In [8]:
!pip install tensorflow
!pip install numpy<2
!pip install --upgrade scipy
!pip install pybind11>=2.12


The system cannot find the file specified.


  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/ea/c2/5ecadc5fcccefaece775feadcd795060adf5c3b29a883bff0e678cfe89af/scipy-1.14.1-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/44.8 MB ? eta -:--:--
    --------------------------------------- 0.6/44.8 MB 17.8 MB/s eta 0:00:03
   - -------------------------------------- 2.1/44.8 MB 26.8 MB/s eta 0:00:02
   -- ------------------------------------- 3.2/44.8 MB 25.8 MB/s eta 0:00:02
   --- ------------------------------------ 4.3/44.8 MB 27.7 MB/s eta 0:00:02
   ---- ----------------------------------- 4.9/44.8 MB 22.3 MB/s eta 0:00:02
   ----- ---------------------------------- 5.8/44.8 MB 21.8 MB/s eta 0:00:02
   ------ --------------------------------- 6.8/44.8 MB 21.8 MB/s eta 0:00:02
   ------ -----------------

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\nsingh\\AppData\\Local\\anaconda3\\Lib\\site-packages\\~cipy\\sparse\\_sparsetools.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import tkinter
import time


In [15]:
from PIL import ImageTk, Image

root = tkinter.Tk()
#Setting it up
keno = ImageTk.PhotoImage(Image.open("./keno.png"))
red = ImageTk.PhotoImage(Image.open("./kokkino.png"))
blue = ImageTk.PhotoImage(Image.open("./ble.png"))

In [16]:
def same(a):
    
    for i in range( len(a) ):
        if a[0] != a[i]:
            return ( False )
        
    return ( True )

#reverse state for 2nd agent, so that it can make predictions according to what the 1st agent has learned so far
def rev( a ):
    b = a.copy()
    b[ a == 1 ] = 2
    b[ a == 2 ] = 1
    return ( b.copy() )

In [17]:
class Environment:
    
    def __init__(self):
        self.R, self.C = 6,7
        self.board = np.zeros( (self.R,self.C), dtype='uint8' )
    
    def reset(self):
        for i in range( self.R ):
            for j in range( self.C ):
                self.board[i][j] = 0
    
    def step(self,who,action):
        
        placed = False
        for j in range( self.R-1, -1, -1 ):
            if self.board[j][action] == 0:
                self.board[j][action] = who
                placed = True
                break
        
        return ( placed )
    
    def getState(self):
        return self.board.copy()
    
    def isFinal(self):
        
        #check horizontal
        for i in range( self.R ):
            for j in range( self.C ):
                if self.board[i][j] != 0:
                    x, y = i, j
                    a = []
                    for k in range(4):
                        a.append( self.board[x][y] )
                        y += 1
                        if y >= self.C:
                            break
                    if same( np.array(a) ) and len(a) == 4:
                        return ( a[0] )
                
        #check vertical
        for i in range( self.R ):
            for j in range( self.C ):
                if self.board[i][j] != 0:
                    x, y = i, j
                    a = []
                    for k in range(4):
                        a.append( self.board[x][y] )
                        x += 1
                        if x >= self.R:
                            break
                    if same( np.array(a) ) and len(a) == 4:
                        return ( a[0] )
                    
        #check diagonal
        for i in range( self.R-3 ):
            for j in range( self.C-3 ):
                if self.board[i][j] != 0:
                    x, y = i, j
                    a = []
                    for k in range(4):
                        a.append( self.board[x][y] )
                        x += 1
                        y += 1
                    if same( np.array(a) ):
                        return ( a[0] )
                    
        #check diagonal
        for i in range( self.R-3 ):
            for j in range( self.C-3 ):
                if self.board[i][j] != 0:
                    x, y = i, j
                    a = []
                    for k in range(4):
                        a.append( self.board[x][y] )
                        x -= 1
                        y -= 1
                    if same( np.array(a) ):
                        return ( a[0] )
                    
        for i in range( self.R ):
            for j in range( self.C ):
                if self.board[i][j] == 0:
                    return ( False )
                    
        return ( True )

In [18]:
env = Environment()
env.board
env.board[1][1]
x, y = 1,1#, j
a = []
for k in range(4):

    print(x,y)
    a.append( env.board[x][y] )
    x += 1
    y += 1
    
print(a)

1 1
2 2
3 3
4 4
[np.uint8(0), np.uint8(0), np.uint8(0), np.uint8(0)]


In [ ]:
#begin DQN
class HiddenLayer:
    def __init__(self, M1, M2, f=tf.nn.tanh, use_bias=True):
        self.W = tf.Variable(tf.random_normal(shape=(M1, M2)))
        self.params = [self.W]
        self.use_bias = use_bias
        if use_bias:
            self.b = tf.Variable(np.zeros(M2).astype(np.float32))
            self.params.append(self.b)
        self.f = f

    def forward(self, X):
        if self.use_bias:
            a = tf.matmul(X, self.W) + self.b
        else:
            a = tf.matmul(X, self.W)
        return self.f(a)


class DQN:
    def __init__(self, D, K, hidden_layer_sizes, gamma, max_experiences=100000, min_experiences=2000, batch_sz=32):
        self.K = K

        # create the graph
        self.layers = []
        M1 = D
        for M2 in hidden_layer_sizes:
            layer = HiddenLayer(M1, M2)
            self.layers.append(layer)
            M1 = M2

        # final layer
        layer = HiddenLayer(M1, K, lambda x: x)
        self.layers.append(layer)

        # collect params for copy
        self.params = []
        for layer in self.layers:
            self.params += layer.params

        # inputs and targets
        self.X = tf.placeholder(tf.float32, shape=(None, D), name='X')
        self.G = tf.placeholder(tf.float32, shape=(None,), name='G')
        self.actions = tf.placeholder(tf.int32, shape=(None,), name='actions')

        # calculate output and cost
        Z = self.X
        #Z = tf.contrib.layers.flatten(self.X)
        for layer in self.layers:
            Z = layer.forward(Z)
        Y_hat = Z
        self.predict_op = Y_hat

        selected_action_values = tf.reduce_sum(
          Y_hat * tf.one_hot(self.actions, K),
          reduction_indices=[1]
        )

        cost = tf.reduce_sum(tf.square(self.G - selected_action_values))
        self.train_op = tf.train.AdamOptimizer(1e-2).minimize(cost)
        # self.train_op = tf.train.AdagradOptimizer(1e-2).minimize(cost)
        # self.train_op = tf.train.MomentumOptimizer(1e-3, momentum=0.9).minimize(cost)
        # self.train_op = tf.train.GradientDescentOptimizer(1e-4).minimize(cost)

        # create replay memory
        self.experience = {'s': [], 'a': [], 'r': [], 's2': [], 'done': []}
        self.max_experiences = max_experiences
        self.min_experiences = min_experiences
        self.batch_sz = batch_sz
        self.gamma = gamma

    def set_session(self, session):
        self.session = session

    def copy_from(self, other):
        # collect all the ops
        ops = []
        my_params = self.params
        other_params = other.params
        for p, q in zip(my_params, other_params):
            actual = self.session.run(q)
            op = p.assign(actual)
            ops.append(op)
        # now run them all
        self.session.run(ops)

    def predict(self, X):
        X = np.atleast_2d(X)
        return self.session.run(self.predict_op, feed_dict={self.X: X})

    def train(self, target_network):
        # sample a random batch from buffer, do an iteration of GD
        if len(self.experience['s']) < self.min_experiences:
            # don't do anything if we don't have enough experience
            return

        # randomly select a batch
        idx = np.random.choice(len(self.experience['s']), size=self.batch_sz, replace=False)
        # print("idx:", idx)
        states = [self.experience['s'][i] for i in idx]
        actions = [self.experience['a'][i] for i in idx]
        rewards = [self.experience['r'][i] for i in idx]
        next_states = [self.experience['s2'][i] for i in idx]
        dones = [self.experience['done'][i] for i in idx]
        next_Q = np.max(target_network.predict(next_states), axis=1)
        targets = [r + self.gamma*next_q if not done else r for r, next_q, done in zip(rewards, next_Q, dones)]

        # call optimizer
        self.session.run(
          self.train_op,
          feed_dict={
            self.X: states,
            self.G: targets,
            self.actions: actions
          }
        )

    def add_experience(self, s, a, r, s2, done):
        if len(self.experience['s']) >= self.max_experiences:
            self.experience['s'].pop(0)
            self.experience['a'].pop(0)
            self.experience['r'].pop(0)
            self.experience['s2'].pop(0)
            self.experience['done'].pop(0)
            self.experience['s'].append(s)
            self.experience['a'].append(a)
            self.experience['r'].append(r)
            self.experience['s2'].append(s2)
            self.experience['done'].append(done)

    def sample_action(self, x, eps):
        if np.random.random() < eps:
            return np.random.choice(self.K)
        else:
            X = np.atleast_2d(x)
        return np.argmax(self.predict(X)[0])

#end dqn
def play_one(env, model, tmodel, eps, gamma, copy_period, learn=True):
  

    global iters
    env.reset()

    observation = env.board.copy().flatten()
    prev_observation = observation.copy().flatten()
        
    done = False

    totalreward = 0
    
    while not done:

        reward = 0
        
        while True:
            #until a valid action is given
            #action = int( input('Player 1 input') )
            
            action = model.sample_action(observation, eps)
            
            placed = env.step(who=1, action=action)
            if placed == True:
                break

        #move has been done, environment has moved to next state, so update
        prev_observation = observation.copy().flatten()
        observation = env.board.copy().flatten()
                
        if env.isFinal() > 0:
            done = True
            reward = +1
            # update the model
            
            model.add_experience(prev_observation, action, reward, observation, done)
            model.train(tmodel)
            iters += 1
            if iters % copy_period == 0:
                tmodel.copy_from(model)
            break

        #print(env.board)

        while True:
            #until a valid action is given
            #action = int( input('Player 2 input') )
            
            #agent 2 has to take an action based on the "reversed state"
            #reversed state: every 1 becomes 2 and vice versa
            action = model.sample_action( rev(observation), eps)
            
            placed = env.step(who=2, action=action)
            if placed == True:
                break

        #move has been done, environment has moved to next state, so update
        prev_observation = observation.copy().flatten()
        observation = env.board.copy().flatten()
        
        if env.isFinal() > 0:
            done = True
            reward = -1
            # update the model
            
            if learn:
                model.add_experience(prev_observation, action, reward, observation, done)
                model.train(tmodel)
                iters += 1
                if iters % copy_period == 0:
                    tmodel.copy_from(model)
            break

        #print(env.board)
        
        observation = env.board.copy().flatten()
        
        # update the model
        if learn:
            model.add_experience(prev_observation, action, reward, observation, done)
            model.train(tmodel)
            iters += 1
            if iters % copy_period == 0:
                tmodel.copy_from(model)

        
    return reward
gamma = 0.99
copy_period = 50

D = 6*7#len(env.observation_space.sample())
K = 7#env.action_space.n
sizes = [512,256,128,64,64]
model = DQN(D, K, sizes, gamma)
tmodel = DQN(D, K, sizes, gamma)
init = tf.global_variables_initializer()
session = tf.InteractiveSession()
session.run(init)
model.set_session(session)
tmodel.set_session(session)


iters = 1

N = 1500
totalrewards = np.empty(N)
costs = np.empty(N)
avg = []

begin = time.time()

for n in range(N):
    eps = 0.1#1.0/np.sqrt(n+1)
    totalreward = play_one(env, model, tmodel, eps, gamma, copy_period)
    totalrewards[n] = totalreward
    #if n % 100 == 0:
    mean_reward = totalrewards[max(0, n-100):(n+1)].mean()
    avg.append( mean_reward )
    print("episode:", n, "total reward:", totalreward, "eps:", eps, "avg reward (last 100):", mean_reward )

    #print( env.board)
    
print("avg reward for last 100 episodes:", totalrewards[-100:].mean())
print("total steps:", totalrewards.sum())

end = time.time()

print( end - begin )

AttributeError: module 'tensorflow' has no attribute 'placeholder'